#### Eval

In [5]:
from ultil_re import *
from typing import Dict
from transformers import AutoConfig, AutoTokenizer, AutoModelForTokenClassification, InputFeatures, AutoModelForSequenceClassification
import numpy as np
from tqdm import tqdm
import torch
from torch.utils.data import DataLoader
device= torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
# model1 sử dụng để predict có relation hay không
model_path1= '/media/data3/users/longnd/ehr-relation-extraction/biobert_re/output/PubmedBert_Task1_'
# model2 sử dụng để predict loại relation

model_path2= '/media/data3/users/longnd/ehr-relation-extraction/biobert_re/output/PubmedBert_Task2_alpha(0.35)'
tokenize_path= '/media/data3/users/longnd/ehr-relation-extraction/biobert_re/model/PubmedBertTokenizer'

label1= ['No_Relation', 'Relation']
label2= ['Association', 'Bind', 'Comparison', 'Conversion', 'Cotreatment', 'Drug_Interaction', 'Negative_Correlation', 'Positive_Correlation']
label_map1: Dict[int, str] = {i: label for i, label in enumerate(label1)}
label_map2: Dict[int, str] = {i: label for i, label in enumerate(label2)}
num_label1= len(label1)
num_label2= len(label2)

config1 = AutoConfig.from_pretrained(model_path1, num_labels=num_label1, 
                                    id2label=label_map1, label2id={label: i for i, label in enumerate(label1)})
tokenizer = AutoTokenizer.from_pretrained(tokenize_path)

model1 = AutoModelForSequenceClassification.from_pretrained(model_path1, config=config1)

config2 = AutoConfig.from_pretrained(model_path2, num_labels=num_label2, 
                                    id2label=label_map2, label2id={label: i for i, label in enumerate(label2)})

model2 = AutoModelForSequenceClassification.from_pretrained(model_path2, config=config2)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [6]:
# input của model đã được xử lý
feature= open_pickle('/media/data3/users/longnd/ehr-relation-extraction/biobert_re/data/PubmedBert_Process/Test.pkl')

In [7]:
class InputFeatures:
    """
    A single set of features of data.
    Property names are the same names as the corresponding inputs to a model.
    """
    def __init__(self, input_ids: List[int], attention_mask: List[int], token_type_ids: Optional[List[int]] = None, 
                 label_ids: Optional[int] = None   ):
        self.input_ids= input_ids # list of token ids
        self.attention_masks= attention_mask # list of attention mask
        self.token_type_ids= token_type_ids # list of token type ids
        self.labels= label_ids # list of label ids
class REDatasetForPredict(Dataset):
    """
    Dataset for relation extraction
    """
    features: List[InputFeatures]
    pad_token_label_id: int = nn.CrossEntropyLoss().ignore_index
    def __init__(self, features: List[InputFeatures]):
        self.features = features
        self.input_ids = [torch.tensor(example.input_ids).long() for example in self.features]
        self.attention_masks = [torch.tensor(example.attention_mask).float() for example in self.features]
        self.token_type_ids = [torch.tensor(example.token_type_ids).long() for example in self.features]
        # self.labels = [torch.tensor(example.labels).long() for example in self.features]

    def __len__(self):
        return len(self.features)

    def __getitem__(self, index):
        return {
            'input_ids': self.input_ids[index],
            'attention_masks': self.attention_masks[index],
            'token_type_ids': self.token_type_ids[index]
            }

def convert_raw_text(text:str, list_en1: List[str], list_en2: List[str]):
    """"""
    list_en= list_en1+ list_en2
    list_en= sorted(list_en, key= lambda x: x[0])   
    new_text= ''
    begin= 0
    for i in list_en:
        # (start, end, type, ent_text)
        assert text[i[0]:i[1]] == i[3], 'start and end of entity is not correct'
        new_text+= text[begin:i[0]]
        new_text+= '@'+i[2]+'$'+' '+ i[3]+' '+'@/'+i[2]+'$'
        begin= i[1]
    new_text+= text[begin:]
    return new_text

def convert_text_to_input_bert(text, entity_list, tokenizer, max_seq_length=512):
    relation_pair= [['GeneOrGeneProduct', 'GeneOrGeneProduct'], ['GeneOrGeneProduct', 'DiseaseOrPhenotypicFeature'],
                    ['GeneOrGeneProduct', 'ChemicalEntity'], ['DiseaseOrPhenotypicFeature', 'SequenceVariant'],
                    ['ChemicalEntity', 'DiseaseOrPhenotypicFeature'], ['ChemicalEntity', 'ChemicalEntity'],
                    ['ChemicalEntity', 'SequenceVariant'], ['SequenceVariant', 'SequenceVariant'],
                    ['GeneOrGeneProduct', 'SequenceVariant']]
    feature= []
    # entity_list: {code: (N, start, end, type, ent_text)}
    code_list = []
    keys= list(entity_list.keys())
    for i in range(len(keys)):
            entity1= entity_list[i]
            ent1= entity1[0][2]
            for j in range(i, len(keys)):
                entity2= entity_list[j]
                ent2= entity2[0][2]
                code_list.append(keys[i], keys[j])
                if ((ent1, ent2) not in relation_pair) and ((ent1, ent2) not in relation_pair):
                    continue
                sequence= convert_raw_text(text, entity1, entity2)
                encoded_input = tokenizer.encode_plus(
                                sequence,
                                add_special_tokens=True,
                                truncation=True,
                                padding="max_length",
                                max_length= max_seq_length,
                                return_attention_mask=True,
                                return_token_type_ids=True)
                input_ids = encoded_input["input_ids"]
                token_type_ids = encoded_input["token_type_ids"]
                attention_mask = encoded_input["attention_mask"]
                feature.append(InputFeatures(input_ids=input_ids, attention_mask=attention_mask, token_type_ids= token_type_ids, label_ids= 0)) 
    return feature, code_list
 


In [8]:
test_dataset= REDatasetForPredict(features= feature)
data = DataLoader(test_dataset, batch_size= 1, shuffle= False, num_workers= 6)

In [57]:
def predict(model1, model2, data_loader, relation_pair, device='cuda:0'):
    '''Model 1: no relation and relation'''
    model1 = model1.to(device)
    model2 = model2.to(device)
    # batch_preds = np.empty((0,), dtype=np.int64)
    batch_preds = []
    pos_pair= {'G-G':['Association','Bind', 'Negative_Correlation','Positive_Correlation', 'No_Relation'],
            'G-D':['Association', 'Negative_Correlation','Positive_Correlation', 'No_Relation'],
            'C-D':['Association', 'Negative_Correlation','Positive_Correlation', 'No_Relation'],
            'C-C':['Association', 'Bind', 'Comparison', 'Conversion', 'Cotreatment', 'Drug_Interaction', 
                        'Negative_Correlation', 'Positive_Correlation', 'No_Relation'],
            'G-C':['Association','Bind', 'Negative_Correlation','Positive_Correlation', 'No_Relation'],
            'D-V':['Association', 'Negative_Correlation','Positive_Correlation', 'No_Relation'],
            'G-V':['No_Relation'],
            'V-V':['Association', 'No_Relation'],
            'C-V':['Association', 'Negative_Correlation','Positive_Correlation', 'No_Relation']}
    label= ['Association', 'Bind', 'Comparison', 'Conversion', 'Cotreatment', 'Drug_Interaction', 'Negative_Correlation', 
                'Positive_Correlation', 'No_Relation']
    label2id = {l:i for i, l in enumerate(label)}
    pos_pair2id = {}
    for k, v in pos_pair.items():
        pos_pair2id[k] = [label2id[i] for i in v if label2id[i]!= 8]
    
    for idx, batch in enumerate(tqdm(data_loader)):
        with torch.no_grad():
            outputs = model1(batch['input_ids'].to(device), batch['attention_masks'].to(device), batch['token_type_ids'].to(device))
            preds = np.argmax(outputs[0].detach().cpu().numpy(), axis=1)
            if preds[0] == 0:
                preds= 8
            else:
                outputs = model2(batch['input_ids'].to(device), batch['attention_masks'].to(device), batch['token_type_ids'].to(device))
                logits= outputs[0][0].detach().cpu().numpy()#[pos_pair2id[relation_pair[idx]]]
                preds = np.argmax(logits, axis=0)
            batch_preds.append(preds)         
    return batch_preds


In [58]:
labels= get_labels()
labels_map= {i: label for i, label in enumerate(labels)}
pred= predict(model1, model2, data, relation_pair)
prediction= [labels_map[i] for i in pred]

100%|██████████| 8433/8433 [02:24<00:00, 58.54it/s]


In [59]:
relation_pair= get_pair_relation(pubtator_file= '/media/data3/users/longnd/ehr-relation-extraction/biobert_re/data/BioRED/Test.PubTator')

In [60]:
label_test= []
for i in feature:
    label_test.append(i.label)

In [61]:
assert len(pred) == len(relation_pair) == len(label_test)

In [62]:
set(relation_pair)

{'C-C', 'C-D', 'C-V', 'D-V', 'G-C', 'G-D', 'G-G', 'G-V', 'V-V'}

In [63]:
print(get_labels())

['Association', 'Bind', 'Comparison', 'Conversion', 'Cotreatment', 'Drug_Interaction', 'Negative_Correlation', 'Positive_Correlation', 'No_Relation']


In [64]:
import pandas as pd
pd.DataFrame(relation_pair).value_counts()

G-G    3470
G-D    1263
C-D     694
C-C     670
G-C     664
D-V     557
G-V     544
V-V     404
C-V     167
Name: count, dtype: int64

In [65]:
# tp, p_true, p_pred
a= {i: [0, 0, 0] for i in set(relation_pair)}
for ent in a.keys():
    for i in range(len(label_test)):
        if label_test[i]== pred[i] and relation_pair[i]== ent and label_test[i]!= 8:
            a[ent][0]+= 1
        if label_test[i]!= 8 and relation_pair[i]== ent:
            a[ent][1]+= 1
        if pred[i]!= 8 and relation_pair[i]== ent:
            a[ent][2]+= 1

In [66]:
for i in a.keys():
    tp, p_true, p_pred= a[i]
    
    pre= tp/p_pred
    rec= tp/p_true
    print(i)
    print('     precision: ', pre)
    print('     recall: ', rec)
    print('     f1: ', 2*pre*rec/(pre+rec))

C-C
     precision:  0.5967741935483871
     recall:  0.5522388059701493
     f1:  0.5736434108527133
G-G
     precision:  0.34935897435897434
     recall:  0.29222520107238603
     f1:  0.3182481751824817
G-C
     precision:  0.45098039215686275
     recall:  0.5054945054945055
     f1:  0.4766839378238342
G-V
     precision:  0.42592592592592593
     recall:  0.3709677419354839
     f1:  0.39655172413793105
V-V
     precision:  0.2857142857142857
     recall:  0.23529411764705882
     f1:  0.2580645161290323
C-V
     precision:  0.30434782608695654
     recall:  0.19444444444444445
     f1:  0.23728813559322037
C-D
     precision:  0.5703125
     recall:  0.474025974025974
     f1:  0.5177304964539007
G-D
     precision:  0.43037974683544306
     recall:  0.38636363636363635
     f1:  0.40718562874251496
D-V
     precision:  0.4166666666666667
     recall:  0.3488372093023256
     f1:  0.379746835443038
